<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Importar os módulos

In [24]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import Dataset_transf as dprep
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Obter os dataframes dos ficheiros

In [2]:
failures_df, signals_df, logs_df = dprep.get_data()

In [3]:
df_list = [failures_df, signals_df, logs_df]

# Criação de datasets de components

In [4]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = dprep.component_df_creation(signals_df)

In [5]:
comp_df_list = [df_generator, df_hydraulic, df_gen_bear, df_transformer,  df_gearbox]

In [12]:
component_list = failures_df.Component.unique()
component_list

array(['GENERATOR', 'HYDRAULIC_GROUP', 'GENERATOR_BEARING', 'TRANSFORMER',
       'GEARBOX'], dtype=object)

# Merge dos DF com o DF de failures

In [10]:
for i in range(len(component_list)):
    comp_df_list[i] = dprep.sig_fail_merge_dfs(sig_df=comp_df_list[i],
                                               fail_df=failures_df,
                                               component=component_list[i])

In [22]:
comp_df_list[0]

,Amb_Temp_Avg,Amb_WindDir_Abs_Avg,Amb_WindDir_Relative_Avg,Amb_WindSpeed_Avg,Amb_WindSpeed_Est_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Std,Blds_PitchAngle_Avg,Blds_PitchAngle_Max,...,Prod_LatestAvg_ReactPwrGen0,Prod_LatestAvg_ReactPwrGen1,Rtr_RPM_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Std,Spin_Temp_Avg,TTF,Timestamp,Turbine_ID
0,18.0,199.1,-7.8,5.3,5.4,8.6,2.6,0.7,-1.4,0.9,...,0.0,-13521.0,11.3,11.9,10.9,0.2,20.0,63.0,2016-01-01 00:00:00,T11
1,18.0,207.5,0.6,5.7,5.8,11.1,1.7,0.9,-1.7,-0.8,...,0.0,-15363.0,11.5,12.4,11.0,0.4,20.0,63.0,2016-01-01 00:10:00,T11
2,18.0,190.5,-16.5,6.1,6.1,9.9,1.5,0.8,-1.9,-1.3,...,0.0,-15724.0,11.9,12.6,11.3,0.2,20.0,63.0,2016-01-01 00:20:00,T11
3,18.0,214.6,7.6,6.3,6.5,9.4,2.7,0.8,-2.0,0.3,...,0.0,-16447.0,12.2,13.0,11.2,0.4,20.0,63.0,2016-01-01 00:30:00,T11
4,18.0,212.1,5.2,6.4,6.3,11.6,1.7,0.9,-2.0,-1.0,...,0.0,-16343.0,12.4,13.6,11.1,0.6,20.0,63.0,2016-01-01 00:40:00,T11
5,18.0,212.1,5.2,6.2,6.1,21.7,1.9,0.8,-2.0,-0.8,...,0.0,-16605.0,12.1,13.2,11.1,0.5,20.0,63.0,2016-01-01 00:50:00,T11
6,18.0,218.4,11.5,5.5,5.4,8.5,2.7,0.8,-1.5,0.2,...,0.0,-13636.0,11.3,12.5,11.0,0.2,20.0,63.0,2016-01-01 01:00:00,T11
7,18.0,194.9,-12.0,5.0,5.2,7.7,1.6,0.7,-1.1,0.3,...,0.0,-13071.0,11.1,11.4,10.9,0.1,20.0,63.0,2016-01-01 01:10:00,T11
8,18.0,219.5,12.6,5.2,5.2,8.8,1.7,0.8,-1.3,0.3,...,0.0,-13548.0,11.2,11.6,10.9,0.1,20.0,63.0,2016-01-01 01:20:00,T11
9,18.0,194.4,-21.2,4.9,4.9,7.3,2.1,0.7,-1.0,1.3,...,0.0,-13590.0,11.1,11.3,10.9,0.1,20.0,63.0,2016-01-01 01:30:00,T11


# Fillna by turbine

In [15]:
turbine_list = signals_df.Turbine_ID.unique()
turbine_list

array(['T11', 'T06', 'T01', 'T09', 'T07'], dtype=object)

In [20]:
for i in range(len(component_list)):
    comp_df_list[i] = dprep.fill_na_by_turbine(comp_df_list[i], turbine_list)

C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\notebooks\Dataset_transf.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  index = index[['date','Timestamp', 'Turbine_ID']]
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\notebooks\Dataset_transf.py:168: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\Users\ricar\code\RicardoJPedro82\wmillfailprev\notebooks\Dataset_transf.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

# Criação de Variável alvo

In [27]:
for i in range(len(comp_df_list)):
    comp_df_list[i] = comp_df_list[i].apply(lambda x: dprep.Failure_Time_Horizon(x['TTF'], 60),axis=1)

In [32]:
comp_df_list

[0        0
 1        0
 2        0
 3        0
 4        0
 5        0
 6        0
 7        0
 8        0
 9        0
 10       0
 11       0
 12       0
 13       0
 14       0
 15       0
 16       0
 17       0
 18       0
 19       0
 20       0
 21       0
 22       0
 23       0
 24       0
 25       0
 26       0
 27       0
 28       0
 29       0
         ..
 87177    0
 87178    0
 87179    0
 87180    0
 87181    0
 87182    0
 87183    0
 87184    0
 87185    0
 87186    0
 87187    0
 87188    0
 87189    0
 87190    0
 87191    0
 87192    0
 87193    0
 87194    0
 87195    0
 87196    0
 87197    0
 87198    0
 87199    0
 87200    0
 87201    0
 87202    0
 87203    0
 87204    0
 87205    0
 87206    0
 Length: 434147, dtype: int64,
 0        0
 1        0
 2        0
 3        0
 4        0
 5        0
 6        0
 7        0
 8        0
 9        0
 10       0
 11       0
 12       0
 13       0
 14       0
 15       0
 16       0
 17       0
 18       0
 19      